### Week 3 Assignment - Segmenting and Clustering Neighborhoods in Toronto

#### Yichuan Wang

In [2]:
# import needed packages
import pandas as pd
import numpy as np
import requests

##### Part 1. Read the Toronto neighborhood table from Wikipedia 

In [37]:
# website where the table is located
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

# use pandas to find the tables from website above
list_of_df=pd.read_html(url, header=0)

# the first table is the one we need
toronto_df=list_of_df[0]
toronto_df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


We need to drop entries where Boroughs are 'Not assigned' and replace 'Not assigned' Neighborhoods with the Borough names.

In [39]:
# first change the spelling of Neighbourhood to Neighborhood
toronto_df.columns=['Postal Code','Borough','Neighborhood']

# drop 'Not assigned' Boroughs
toronto_df=toronto_df[toronto_df['Borough']!='Not assigned']

# replace 'Not assigned' neighborhoods with Borough names
idx=toronto_df['Neighborhood']=='Not assigned'
toronto_df['Neighborhood'][idx]=toronto_df['Borough'][idx]

# reset the table index
toronto_df=toronto_df.reset_index(drop=True)

# print a summary view of the table
print(toronto_df.head())
print('\nThe table size is: ',toronto_df.shape)
print('\n',toronto_df.describe())

  Postal Code           Borough                                 Neighborhood
0         M3A        North York                                    Parkwoods
1         M4A        North York                             Victoria Village
2         M5A  Downtown Toronto                    Regent Park, Harbourfront
3         M6A        North York             Lawrence Manor, Lawrence Heights
4         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government

The table size is:  (103, 3)

        Postal Code     Borough Neighborhood
count          103         103          103
unique         103          10           99
top            M2J  North York    Downsview
freq             1          24            4


Since all postal codes are unique in the table, we do not need to combine any entries. A quick inspection of the table on the web shows there's no missing entry in the Postal Code column.

##### Part 2. Obtain the coordinates for each neighborhood

I'm using the CSV file provided to obtain the coordinates.

In [43]:
# read the coordinate table from CSV file
coord_df=pd.read_csv('Geospatial_Coordinates.csv')

# merge the two tables
nbhd_df=pd.merge(toronto_df, coord_df, on='Postal Code')

nbhd_df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
